In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The goal of this project is to create a website where people can find their estimated house prices by providing relavent information. 
This notebook is used for data visualization, helping us to understand the data and choose a good model.

This notebook is inspired by and mostly based on Ruchi Bhatia's work: [Housing Prices: EDA, Permutation Imp, Partial plot](http://https://www.kaggle.com/ruchi798/housing-prices-eda-permutation-imp-partial-plot)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from geopy.geocoders import Nominatim

In [ ]:
#!pip install geopy
#!pip install Nominatim

In [ ]:
Mumbai_df = pd.read_csv('../input/housing-prices-in-metropolitan-areas-of-india/Mumbai.csv')
Delhi_df = pd.read_csv('../input/housing-prices-in-metropolitan-areas-of-india/Delhi.csv')
Chennai_df = pd.read_csv('../input/housing-prices-in-metropolitan-areas-of-india/Chennai.csv')
Hyderabad_df = pd.read_csv('../input/housing-prices-in-metropolitan-areas-of-india/Hyderabad.csv')

In [ ]:
Mumbai_df.head()

In [ ]:
print(Mumbai_df.shape)
Mumbai_df.describe()

for some reason, number 9 is used to represent if the existence of certain amenity is unknow, so we will just drop them later. Now we will change 9 to np.nan

In [ ]:
Mumbai_df.replace(9, np.nan, inplace=True)
Delhi_df.replace(9, np.nan, inplace=True)
Chennai_df.replace(9, np.nan, inplace=True)
Hyderabad_df.replace(9, np.nan, inplace=True)

In [ ]:
Mumbai_df = Mumbai_df.dropna()
Delhi_df = Delhi_df.dropna()
Chennai_df = Chennai_df.dropna()
Hyderabad_df = Hyderabad_df.dropna()

In [ ]:
print(f"Mumbai:{Mumbai_df.shape}\n")
print(f"Mumbai:{Delhi_df.shape}\n")
print(f"Mumbai:{Chennai_df.shape}\n")
print(f"Mumbai:{Hyderabad_df.shape}\n")

In [ ]:
print(f"Data types of all columns: \n{Mumbai_df.dtypes}")

write down the column names for amentities for future analysis.

In [ ]:
amentities = ["Wardrobe","Refrigerator","Sofa","DiningTable","TV","GolfCourse","Microwave","BED","LiftAvailable","Children'splayarea","Wifi","AC","Gasconnection","WashingMachine","Hospital","MultipurposeRoom","Cafeteria","StaffQuarter","CarParking","PowerBackup","24X7Security","School","ClubHouse","ATM","SportsFacility","Intercom","ShoppingMall","IndoorGames","RainWaterHarvesting","JoggingTrack","LandscapedGardens","SwimmingPool","Gymnasium","MaintenanceStaff"]

1. Modifying price to price in lakhs(INR) 1 lakhs(INR) == 100,000 rupees

In [ ]:
Mumbai_df['Price'] = Mumbai_df['Price']/100000
Delhi_df['Price'] = Delhi_df['Price']/100000
Chennai_df['Price'] = Chennai_df['Price']/100000
Hyderabad_df['Price'] = Hyderabad_df['Price']/100000

In [ ]:
#Removing Outliers
#Mumbai_df = Mumbai_df[Mumbai_df['Price'] < 40000000]
#Delhi_df = Delhi_df[Delhi_df['Price'] < 4000000]
#Chennai_df = Chennai_df[Chennai_df['Price'] < 400]
#Hyderabad_df = Hyderabad_df[Hyderabad_df['Price'] < 400]

In [ ]:
geolocator = Nominatim(user_agent="visual")

def geogeneration(df):
    lat = []
    long = []
    t = 0
    for i in df['Location']:
        location = geolocator.geocode(i, timeout=None)
        if t%100 == 0:
            print(t)
        t += 1
        try:
            lat.append(location.latitude)
            long.append(location.longitude)
        except:
            lat.append("NA")
            long.append("NA")
    df['Latitude'] = lat
    df['Longitude'] = long

In [ ]:
geogeneration(Mumbai_df)
geogeneration(Delhi_df)
geogeneration(Chennai_df)
geogeneration(Hyderabad_df)

In [ ]:
Mumbai_df.to_csv('/kaggle/working/Mumbai_updated.csv')
Delhi_df.to_csv('/kaggle/working/Delhi_updated.csv')
Chennai_df.to_csv('/kaggle/working/Chennai_updated.csv')
Hyderabad_df.to_csv('/kaggle/working/Hyderabad_updated.csv')

In [ ]:
df1 = pd.read_csv('/kaggle/working/Mumbai_updated.csv')
df2 = pd.read_csv('/kaggle/working/Delhi_updated.csv')
df3 = pd.read_csv('/kaggle/working/Chennai_updated.csv')
df4 = pd.read_csv('/kaggle/working/Hyderabad_updated.csv')

In [ ]:
df1.head()

In [ ]:
df1 = df1.drop(['Unnamed: 0'],axis=1)
df2 = df2.drop(['Unnamed: 0'],axis=1)
df3 = df3.drop(['Unnamed: 0'],axis=1)
df4 = df4.drop(['Unnamed: 0'],axis=1)

In [ ]:
def triple_plot(x,title):
    fig, ax = plt.subplots(3,1,figsize=(20,10),sharex=True)
    sns.distplot(x,ax=ax[0],bins=150)
    #ax[0].set(xlabel=None)
    ax[0].set_title('Histogram + KDE')
    sns.boxplot(x,ax=ax[1])
    #ax[1].set(xlabel=None)
    ax[1].set_title('Boxplot')
    sns.violinplot(x,ax=ax[2])
    ax[2].set(xlabel=None)
    ax[2].set_title('Violin plot')
    fig.suptitle(title,fontsize=16)
    #plt.tight_layout(pad=3.0)
    plt.xlim(0,1000)
    plt.show()

triple_plot(df1['Price'],'Distribution of Price(in lakhs) in Mumbai')

In [ ]:
triple_plot(df2['Price'],'Distribution of Price(in lakhs) in Delhi')
triple_plot(df3['Price'],'Distribution of Price(in lakhs) in Chennai')
triple_plot(df4['Price'],'Distribution of Price(in lakhs) in Hyderabad')

based on the plots for prices, we see that this dataset has majority of its data points at price under 200 lakhs(INR). For the purpose of our project, we should limit the prices beyond 200 lakhs.

In [ ]:
corr_matrix = df1.corr()

In [ ]:
plt.figure(figsize=(28, 16))
sns.heatmap(data=corr_matrix, annot=True)

In [ ]:
grid = sns.PairGrid(df1)
grid.map_diag(sns.distplot)
grid.map_offdiag(sns.scatterplot)

the image is hard to see, but basically it doesn't provide much information because most of the columns are 0,1. Now I want to investigate if properties having more amenties means they are more expensive. I will sum up all the columns and call it "NumOfAmentities"

In [ ]:
df1['NumOfAmentities'] = df1[amentities].sum(axis=1)
df2['NumOfAmentities'] = df2[amentities].sum(axis=1)
df3['NumOfAmentities'] = df3[amentities].sum(axis=1)
df4['NumOfAmentities'] = df4[amentities].sum(axis=1)

In [ ]:
sns.scatterplot(x="NumOfAmentities", y="Price", data=df1,marker="P")
plt.title('Number of Amentities vs Price in Mumbai',size=16)
plt.gcf().set_size_inches(6,8)
plt.show()

Seems like my hypothesis is wrong.

In [ ]:
#Import modules from sklearn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_regression

#Create an instance of StandardScaler and LinearRegression
scaler = StandardScaler()
reg = LinearRegression()

#Summary of Data
df1.describe()

In [ ]:
#Create independent and dependent variables of Mumbai
x1 = df1[['Area', 'No. of Bedrooms', 'NumOfAmentities']]
y1 = df1[['Price']]

#Scaling Inputs
scaler.fit(x1)
x_scaled1 = scaler.transform(x1)

#Regression
reg.fit(x_scaled1, y1)

#Summary table
reg_summary = pd.DataFrame([['Area'],['No. of Bedrooms'],['NumOfAmentities']], columns = ['Features'])
reg_summary['Coefficients'] = reg.coef_[0,0], reg.coef_[0,1], reg.coef_[0,2]
f = f_regression(x_scaled1,y1)
reg_summary['P-Values'] = f[1][0], f[1][1],f[1][2]
reg_summary

In [ ]:
#Create independent and dependent variables of Delhi
x2 = df2[['Area', 'No. of Bedrooms', 'NumOfAmentities']]
y2 = df2[['Price']]

#Scaling Inputs
scaler.fit(x2)
x_scaled2 = scaler.transform(x2)

#Regression
reg.fit(x_scaled2, y2)

#Summary table
reg_summary = pd.DataFrame([['Area'],['No. of Bedrooms'],['NumOfAmentities']], columns = ['Features'])
reg_summary['Coefficients'] = reg.coef_[0,0], reg.coef_[0,1], reg.coef_[0,2]
f2 = f_regression(x_scaled2,y2)
reg_summary['P-Values'] = f2[1][0], f2[1][1],f2[1][2]
reg_summary

In [ ]:
#Create independent and dependent variables of Chennai
x3 = df3[['Area', 'No. of Bedrooms', 'NumOfAmentities']]
y3 = df3[['Price']]

#Scaling Inputs
scaler.fit(x3)
x_scaled3 = scaler.transform(x3)

#Regression
reg.fit(x_scaled3, y3)

#Summary table
reg_summary = pd.DataFrame([['Area'],['No. of Bedrooms'],['NumOfAmentities']], columns = ['Features'])
reg_summary['Coefficients'] = reg.coef_[0,0], reg.coef_[0,1], reg.coef_[0,2]
f3 = f_regression(x_scaled3,y3)
reg_summary['P-Values'] = f3[1][0], f3[1][1],f3[1][2]
reg_summary

In [ ]:
#Create independent and dependent variables of Hyderabad
x4 = df4[['Area', 'No. of Bedrooms', 'NumOfAmentities']]
y4 = df4[['Price']]

#Scaling Inputs
scaler.fit(x4)
x_scaled4 = scaler.transform(x4)

#Regression
reg.fit(x_scaled4, y4)

#Summary table
reg_summary = pd.DataFrame([['Area'],['No. of Bedrooms'],['NumOfAmentities']], columns = ['Features'])
reg_summary['Coefficients'] = reg.coef_[0,0], reg.coef_[0,1], reg.coef_[0,2]
f4 = f_regression(x_scaled4,y4)
reg_summary['P-Values'] = f4[1][0], f4[1][1],f4[1][2]
reg_summary

In [ ]:
# Import the module for the split
from sklearn.model_selection import train_test_split

# Split the variables with an 80-20 split and some random state
# To have the same split as mine, use random_state = 365
x_train, x_test, y_train, y_test = train_test_split(x_scaled1, y1, test_size=0.2, random_state=365)

In [ ]:
# Create a linear regression object
reg = LinearRegression()
# Fit the regression with the scaled TRAIN inputs and targets
reg.fit(x_train,y_train)

In [ ]:
# Let's check the outputs of the regression
# I'll store them in y_hat as this is the 'theoretical' name of the predictions
y_hat = reg.predict(x_train)

In [ ]:
# The simplest way to compare the targets (y_train) and the predictions (y_hat) is to plot them on a scatter plot
# The closer the points to the 45-degree line, the better the prediction
plt.scatter(y_train, y_hat)
# Let's also name the axes
plt.xlabel('Targets (y_train)',size=18)
plt.ylabel('Predictions (y_hat)',size=18)
# Sometimes the plot will have different scales of the x-axis and the y-axis
# This is an issue as we won't be able to interpret the '45-degree line'
# We want the x-axis and the y-axis to be the same
plt.xlim(0,2000)
plt.show()

In [ ]:
# Another useful check of our model is a residual plot
# We can plot the PDF of the residuals and check for anomalies
sns.distplot(y_train - y_hat)

# Include a title
plt.title("Residuals PDF", size=18)